# Chess Gameplay

First import our `chess_gameplay` module.

In [1]:
import chess_gameplay as chg

## Playing a game

Let's take a look at how games will be played in the tournament. Before running this cell, open the `demo.png` file alongside so you can watch the action!

In [2]:
# Instantiate agents and record team names. Note we're instantiating Agents with no arguments. These agents have been
# passed no models to inform their selections, so they will play purely random moves.

agents = {'white': chg.Agent(), 'black': chg.Agent()}
teams = {'white': 'Team White', 'black': 'Team Black'}

# Then call the `play_game` function. Note that we're playing here to a maximum depth of 5 moves each. In the
# tournament we will be playing to a maximum depth of 50 moves. We are also passing "poseval"=True which means 
# we will use StockFish to evaluate the board state after each move. These evaluations are used to update the 
# eval bar on the left side of the board rendering in `demo.png`. StockFish will be constrained by a time limit
# of 2 seconds and a depth limit of 25.

game_result = chg.play_game(
    agents, 
    teams, 
    max_moves=10, 
    min_seconds_per_move=2, 
    verbose=True, 
    poseval=True, 
    image_path="demo.png"
)

# Run this cell. A file `demo.pgn` will be saved to this repo directory which you can open and watch as it is 
# updated with moves from the game. The game may end in a checkmate, in which case the winner will recieve 1 point 
# and the loser will receieve 0 points. If the game ends in a draw or a stalemate, both will receieve 0 points. If 
# the maximum number of moves is reached without a conclusion to the game, the StockFish evaluations of the final 
# board state are used as the points for each Agent. For each pairing in the tournament, teams will play once as 
# white and once as black. The winner of the pairing will be the team with the highest score summed over the two 
# games. In the event of a draw, the pairing will be played again until a winner is declared.

white: Nf3
black: a5
white: c3
black: Ra6
white: Na3
black: Rd6
white: g3
black: e6
white: Rb1
black: Nh6
white: Nb5
black: Rd4
white: g4
black: Nxg4
white: Na3
black: e5
white: Ng5
black: Rd3
white: f4
black: c6
Max moves reached.
White score: 0.513, Black score: 0.487


## Agents using models

Until you have trained a model and saved a checkpoint, you will not be able to run the following cell, but you can see how your model will be called and passed to an Agent to play with.

In [4]:
import torch
import yaml

# Your model must be imported exactly as follows; from a module called "model" (a file called "model.py") and with
# the class name "Model".

from competing_models.model_winner import Model as WinnerModel
from competing_models.model_my import Model as MyModel

# All necessary arguments for your model to initialize with must be saved in a YAML file called "model_config.yaml"
# so that your model can be instantiated exactly as follows. Your model must NOT require any initialization arguments
# besides those described in your "model_config.yaml" file.

winner_model_config = yaml.safe_load(open("./competing_models/model_config_winner.yaml"))
winner_model = WinnerModel(**winner_model_config)

# Your model checkpoint must be called "checkpoint.pt" and must be a dictionary-like object with your model weights
# stored at the key "model" so that it can be loaded into your model exactly as follows.

checkpoint_winner = torch.load("/shared/artifacts/hackathon_winner_checkpoints/checkpoints/CHK5/checkpoint.pt", map_location="cpu")
winner_model.load_state_dict(checkpoint_winner["model"])


my_model_config = yaml.safe_load(open("./competing_models/model_config_my.yaml"))
my_model = MyModel(**my_model_config)

checkpoint_my = torch.load("/shared/artifacts/my_experiments/experiment_1/checkpoints/CHK1/checkpoint.pt", map_location="cpu")
my_model.load_state_dict(checkpoint_my["model"])




# Note: when you load your model weights you may see the following warning. You can safely ignore this warning.

ignore = """
/root/.chess/lib/python3.10/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
"""

In [6]:
# The model is passed as the first positional argument to the Agent, and is then available to the agent to use for
# selecting moves in the game.

agents = {'white': chg.Agent(my_model), 'black': chg.Agent(winner_model)}
teams = {'white': 'Team White', 'black': 'Team Black'}

game_result = chg.play_game(
    agents, 
    teams, 
    max_moves=50, 
    min_seconds_per_move=0, 
    verbose=True, 
    poseval=True, 
    image_path="demo.png"
)

white: f3
black: d5
white: a4
black: h5
white: e3
black: Bh3
white: Kf2
black: Bxg2
white: Qe1
black: Nc6
white: Qd1
black: Nd4
white: Ba6
black: Nxc2
white: Bf1
black: Nxa1
white: Kg3
black: Bxh1
white: d4
black: Rc8
white: Nd2
black: Nc2
white: Be2
black: Nxd4
white: b3
black: Nxe2+
white: Qxe2
black: e5
white: f4
black: exf4+
white: Kxf4
black: Rh7
white: a5
black: d4
white: exd4+
black: Qe7
white: Qd3
black: Rb8
white: Qa6
black: bxa6
white: d5
black: Qe2
white: Ba3
black: Qxh2+
white: Kg5
black: Rxb3
white: Ne4
black: Rd3
white: Ng3
black: Re3
white: Bxf8
black: Kd8
white: Kf5
black: Bxd5
white: Kg5
black: c6
white: Bd6
black: Be6
white: Nf3
black: Rh8
white: Bf4
black: Bh3
white: Nd2
black: Rxg3+
white: Bxg3
black: Qxg3#
Checkmate! black wins!
White score: 0.5, Black score: 0.5
